# Using a custom dataset

This notebook demonstrates how to train a model using a custom dataset.

In [ ]:
import os

# These environment variables control where
# training and eval logs are written.
os.environ["RUN_DIR"] = "runs"
os.environ["EVAL_RUN_DIR"] = "eval_runs"

# This is used to set a constant Tensorboard port.
os.environ["TENSORBOARD_PORT"] = str(8989)

import ml.api as ml  # Source: https://github.com/codekansas/ml-starter

# Enables logging to `stdout`.
ml.configure_logging(use_tqdm=True)

# Imports these files to add them to the model and task registry.
from usa.models.point2emb import Point2EmbModel
from usa.tasks.clip_sdf import ClipSdfTask

We're using the built-in config file instead of specifying the full config in code.

For this example, we are using a pre-recorded Record3D file, but you can easily create your own using the [Record3D app](https://record3d.app/). Save the `.r3d` file (the internal file format) to somewhere and point the `dataset_path` to that location.

In [ ]:
import requests
from pathlib import Path
from omegaconf import OmegaConf

data_root = Path("data")
data_root.mkdir(exist_ok=True)
dataset_path = data_root / "dataset.r3d"

# We're downloading an existing dataset, but you can use your own instead.
dataset_url = "https://github.com/codekansas/usa/releases/download/v1.0.0/lab.r3d"
if not dataset_path.exists():
    with requests.get(dataset_url, stream=True) as r:
        r.raise_for_status()
        with open(dataset_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

# Using the default config, but overriding the dataset.
config = OmegaConf.load("config.yaml")
config.task.dataset = "r3d"
config.task.dataset_path = str(dataset_path)

# We still need to explicitly set these variables.
config.trainer.exp_name = "jupyter"
config.trainer.log_dir_name = "test"
config.trainer.base_run_dir = "runs"
config.trainer.run_id = 0

In [ ]:
objs = ml.instantiate_config(config)

# Unpacking the different components.
model = objs.model
task = objs.task
optimizer = objs.optimizer
lr_scheduler = objs.lr_scheduler
trainer = objs.trainer

from tensorboard import notebook

# Show Tensorboard inside the notebook.
notebook.display(port=int(os.environ['TENSORBOARD_PORT']))

# Runs the training loop.
trainer.train(model, task, optimizer, lr_scheduler)